# Defining a Unified Query Interface over your Data

Query Decomposition: The ability to decompose a complex query into a simpler query given the content of the index.

Use ChatGPT as the LLM model

In [ ]:
import logging
import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Uncomment if you want to temporarily disable logger
logger = logging.getLogger()
logger.disabled = True

In [ ]:
from gpt_index import (
    GPTVectorStoreIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    GPTTreeIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext
)
from langchain.llms.openai import OpenAIChat, OpenAI
import requests

#### Load Datasets

Load Wikipedia pages as well as Paul Graham's "What I Worked On" essay

In [ ]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

In [ ]:
from pathlib import Path

import requests
for title in wiki_titles:
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': title,
            'prop': 'extracts',
            # 'exintro': True,
            'explaintext': True,
        }
    ).json()
    page = next(iter(response['query']['pages'].values()))
    wiki_text = page['extract']

    data_path = Path('data')
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", 'w') as fp:
        fp.write(wiki_text)


In [ ]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(input_files=[f"data/{wiki_title}.txt"]).load_data()


### Building each Vector Index
Build a vector index for the wiki pages about cities and persons, and PG essay

In [ ]:
# # LLM Predictor (gpt-3.5-turbo)
llm_predictor_chatgpt = LLMPredictor(llm=OpenAIChat(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor_chatgpt, chunk_size_limit=1024
)


llm_predictor_gpt4 = LLMPredictor(llm=OpenAIChat(temperature=0, model_name="gpt-4"))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor_gpt4, chunk_size_limit=1024
)

In [ ]:
# Build city document index
vector_indices = {}
for wiki_title in wiki_titles:
    # build vector index
    vector_indices[wiki_title] = GPTVectorStoreIndex.from_documents(
        city_docs[wiki_title], service_context=service_context
    )
    # set id for vector index
    vector_indices[wiki_title].index_struct.index_id = wiki_title

In [ ]:
index_summaries = {}
for wiki_title in wiki_titles:
    # set summary text for city
    index_summaries[wiki_title] = (
        f"This content contains Wikipedia articles about {wiki_title}. "
        f"Use this index if you need to lookup specific facts about {wiki_title}.\n"
        "Do not use this index if you want to analyze multiple cities."
    )

#### Test Querying the Vector Index

In [ ]:
query_engine = vector_indices["Toronto"].as_query_engine()
response = query_engine.query("What are the sports teams in Toronto?")

In [ ]:
print(str(response))

### Build a Graph for Compare/Contrast Queries

We compose a keyword table index on top of all the vector indices.
We use this index for compare/contrast queries

In [ ]:
from gpt_index.indices.composability import ComposableGraph

In [ ]:
graph = ComposableGraph.from_indices(
    GPTSimpleKeywordTableIndex,
    [index for _, index in vector_indices.items()], 
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50
)

In [ ]:
# get root index
root_index = graph.get_index(graph.root_id)
# set id of root index
root_index.set_index_id("compare_contrast")
root_summary = (
    "This index contains Wikipedia articles about multiple cities. "
    "Use this index if you want to compare multiple cities. "
)

#### Test querying the graph

In [ ]:
# define decompose_transform
from gpt_index.indices.query.query_transform.base import DecomposeQueryTransform
decompose_transform = DecomposeQueryTransform(
    llm_predictor_chatgpt, verbose=True
)

In [ ]:
# define custom retrievers

from gpt_index.query_engine.transform_query_engine import TransformQueryEngine


custom_query_engines = {}
for index in vector_indices.values():
    query_engine = index.as_query_engine(service_context=service_context)
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine

custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    mode='simple',
    response_mode='tree_summarize',
    service_context=service_context,
    verbose=True,
)

query_engine = graph.as_query_engine(
    custom_query_engines=custom_query_engines
)

In [ ]:
query_str = (
    "Compare and contrast the arts and culture of Houston and Boston. "
)
response = query_engine.query(query_str)

In [ ]:
print(response)

### Build the Outer Router Index

In [ ]:
num_children = len(vector_indices) + 1
outer_graph = ComposableGraph.from_indices(
    GPTTreeIndex,
    [index for _, index in vector_indices.items()] + [root_index], 
    [summary for _, summary in index_summaries.items()] + [root_summary],
    num_children=num_children
)

In [ ]:
# define custom query engines
custom_query_engines = {}
for index in vector_indices.values():
    query_engine = index.as_query_engine(service_context=service_context)
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    mode='simple',
    response_mode='tree_summarize',
    service_context=service_context,
)
custom_query_engines[outer_graph.root_id] = outer_graph.root_index.as_query_engine(
    mode='simple',
    response_mode='tree_summarize',
    service_context=service_context,
)

# define query engine
outer_query_engine = outer_graph.as_query_engine(custom_query_engines=custom_query_engines)

In [ ]:
# ask a compare/contrast question 
response = outer_query_engine.query(
    "Compare and contrast the arts and culture of Houston and Boston.",
)

In [ ]:
str(response)

In [ ]:
response = outer_query_engine.query("What are the sports teams in Toronto?")

In [ ]:
str(response)